In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict
from functools import partial
from itertools import count
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
import string
import re
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
import re
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC,SVC
%matplotlib inline

//anaconda/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# image features cleaning

In [4]:
features_df = pd.read_csv("../../final/data/features_train/features_resnet1000_train.csv", header = None )

In [5]:
features_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,images_train/5373.jpg,-0.899450,-0.930470,-2.503365,-3.172499,-2.819133,0.992159,-3.698863,0.619991,0.956148,...,-3.021916,2.214253,-1.382491,1.672911,1.014233,2.599949,2.773284,-2.066632,0.385754,-3.241320
1,images_train/984.jpg,-1.346954,-3.119461,-0.765971,-1.382550,-1.104675,-3.656271,-4.815436,-0.556942,-1.402286,...,0.011003,-3.968805,-2.694711,-4.196480,-2.880234,-1.210742,-1.605143,-4.859987,-0.837670,-0.967604
2,images_train/7127.jpg,-3.445498,-1.524573,-1.001654,-3.668335,-1.805517,-1.633496,-7.127826,-1.147802,-1.055816,...,-2.991777,-2.628053,-2.971074,-2.537039,-1.707429,1.013672,0.608460,-3.714998,-0.484735,0.138767
3,images_train/9609.jpg,1.114650,-2.167102,0.097881,-1.336255,0.853483,-0.374885,-2.369090,-2.273191,-1.143788,...,-1.248134,-0.633126,-1.723514,-2.638832,0.097149,4.647974,1.030138,-2.193836,1.044024,0.176043
4,images_train/5293.jpg,1.602650,-1.505817,3.029409,4.092412,1.711755,6.271253,4.173686,-2.177313,0.747789,...,-1.285806,-2.266481,-3.898053,2.295787,-1.749552,0.974188,1.258117,-1.975622,-1.278643,-1.941441


In [6]:
features_df2 = features_df.sort([0], ascending = 1)

In [8]:
split= features_df2[0].str.split('/', 1, expand=True)

In [9]:
split.head(5)

,0,1
6643,images_train,0.jpg
2273,images_train,1.jpg
7719,images_train,10.jpg
9839,images_train,100.jpg
9966,images_train,1000.jpg


In [10]:
split2= split[1].str.split('.', 1, expand=True)

In [11]:
del split2[1]

In [13]:
split2 = split2.rename(columns={0: "index"})

In [14]:
result = pd.concat([split2, features_df2], axis=1)

In [15]:
result.head(5)

,index,0,1,2,3,4,5,6,7,8,...,991,992,993,994,995,996,997,998,999,1000
6643,0,images_train/0.jpg,-4.552913,-3.339373,0.739805,-1.064960,-0.385821,-1.374796,-3.172979,-1.216543,...,-2.021026,-4.752773,-4.072814,-3.873384,-3.553856,-2.005920,-2.187191,-4.325321,-1.361008,-0.184895
2273,1,images_train/1.jpg,1.055800,1.345920,-4.621484,-3.391327,-2.474905,-0.765481,-3.783750,-3.479563,...,0.447561,-1.310622,-1.228498,-0.492747,0.690989,1.746380,3.944386,-0.335490,4.469384,-0.193547
7719,10,images_train/10.jpg,-1.601292,-2.199482,-0.254420,-1.865104,-1.763441,0.254424,-1.573081,0.463559,...,-0.095754,-3.099477,-3.825321,0.554782,-0.601681,1.979488,1.848149,-1.287298,-0.238974,-2.143847
9839,100,images_train/100.jpg,-0.596288,-0.154158,2.084059,0.038803,1.274306,2.841651,1.819926,-1.501671,...,-0.568154,0.881252,-0.578756,-1.408792,-0.834128,-0.082579,-0.261926,2.478882,0.054818,0.820606
9966,1000,images_train/1000.jpg,-1.329414,-2.590716,-0.110130,-0.800338,-2.679732,0.630098,-2.879541,1.048122,...,3.146122,-2.295155,-1.682396,-2.140283,-2.799133,-0.214362,0.524082,-2.281344,2.344166,-2.472890


In [16]:
result["index"] = result["index"].astype(float)

In [17]:
result = result.sort(["index"], ascending = 1)

In [18]:
result.head(5)

,index,0,1,2,3,4,5,6,7,8,...,991,992,993,994,995,996,997,998,999,1000
6643,0,images_train/0.jpg,-4.552913,-3.339373,0.739805,-1.064960,-0.385821,-1.374796,-3.172979,-1.216543,...,-2.021026,-4.752773,-4.072814,-3.873384,-3.553856,-2.005920,-2.187191,-4.325321,-1.361008,-0.184895
2273,1,images_train/1.jpg,1.055800,1.345920,-4.621484,-3.391327,-2.474905,-0.765481,-3.783750,-3.479563,...,0.447561,-1.310622,-1.228498,-0.492747,0.690989,1.746380,3.944386,-0.335490,4.469384,-0.193547
8598,2,images_train/2.jpg,-1.323054,-1.990459,-2.646253,-2.555408,0.194242,0.360438,-2.450288,-1.364834,...,-1.727636,-4.679168,-4.418219,-1.780263,-2.594645,-1.229027,0.516662,-3.346858,0.924743,-0.353018
9239,3,images_train/3.jpg,1.642971,-0.591986,-2.008492,-1.683242,0.152383,-0.580315,-2.876101,-2.001029,...,2.253989,-1.823335,-0.135379,-2.881580,-1.125049,1.979692,0.571415,-2.186887,2.552499,2.634879
8410,4,images_train/4.jpg,-0.690964,-3.641853,-1.879540,0.208626,0.236147,-3.154541,-3.594615,0.742241,...,-1.286898,-4.253161,-3.368766,-3.720367,-3.813512,-1.947907,-2.103985,-4.788396,-0.305454,1.142632


# Description BOW

In [19]:
descriptionTrainFolder = "../../final/data/descriptions_train"
featuresTrainFolder = "../../final/data/features_train"
imagesTrainFolder = "../../AML/final/data/images_train"
tagsTrainFolder = "../../AML/final/data/tags_train"

In [20]:
numTrain = 10000
numTest = 2000

In [21]:
def readTagsFile(fileName):
    tags = []
    f = open(fileName,"r")
    for line in f:
        line = line.split(":") # possibly also consider using the keys in each line(vehicle,outdoor etc..)
        tag = line[-1].replace("\n","")
        tags.append(tag)
    return tags

def readDescriptionFile(fileName):
    desc = []
    f = open(fileName,"r")
    for line in f:
        noPuncSentence = stripPunctuation(line)
        desc.append(noPuncSentence)
    return desc

def stripPunctuation(s):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub(' ', s)

In [22]:
def getStopWords():
    return set(stopwords.words("english"))

def lowerCase(x):
    return x.lower()

def lemmatize(lmt,x):
    return lmt.lemmatize(x)

def getPos(line):
    return pos_tag(line)

def tokenize(line):
    return line.split()

def stripPunctuation(s):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub(' ', s)

def stemmer(line):
    stemmer = PorterStemmer()
    stemmedLine = []
    for word in line:
        #if pos == "NN": #only stem nouns
        stem = stemmer.stem(word)
        stemmedLine.append(stem)
    return stemmedLine 

def stemmedDescriptions(lst):
    stemmedDescriptions = []
    for line in lst:
        tokenizedLine = tokenize(line)
        stemmedDescriptions.append(stemmer(tokenizedLine))
    return stemmedDescriptions


In [23]:
def createBag(num,descFolder):
    bag = {}
    stopWords = getStopWords()
    for n in range(num):
        f = descFolder +"/" + str(n) + ".txt"
        fileDescriptions = readDescriptionFile(f) #2D Array
        stemDescriptions = stemmedDescriptions(fileDescriptions) #applying stemming so to aggregate similar words
        for desc in stemDescriptions:
            for word in desc:
                lowerWord = lowerCase(word)
                if str(lowerWord) not in bag.keys() and str(lowerWord) != "" and str(lowerWord) not in stopWords:
                    bag[str(lowerWord)] = 0 
    return bag

def indexBag(bag):
    idx = 0
    for tag in bag.keys():
        bag[tag] = idx
        idx+=1
    return bag

def binaryVectorizeDescriptions(num,descFolder,indexedDescBag):
    stopWords = getStopWords()
    featureVectorList = []
    for n in range(num):
        imageTagVec = [0.0 for tag in range(len(indexedDescBag.keys()))]
        f = descFolder +"/" + str(n) + ".txt"
        fileDescriptions = readDescriptionFile(f) #2D Array
        stemDescriptions = stemmedDescriptions(fileDescriptions) #applying stemming so to aggregate similar words
        for desc in stemDescriptions:
            for word in desc:
                lowerWord = lowerCase(word)
                if str(lowerWord) in indexedDescBag.keys() and str(lowerWord) != "" and str(lowerWord) not in stopWords:
                    imageTagVec[indexedDescBag[str(lowerWord)]] +=1.0
        featureVectorList.append(imageTagVec)
    return pd.DataFrame(featureVectorList,columns = indexedDescBag.keys())

def tfIdfVectorizeDecriptions(num,descFolder):
    """
    this technique will be very sparse - consider using dimensionality reduction here
    """
    tf = TfidfVectorizer(analyzer='word', 
                         ngram_range=(1,1), 
                         min_df = 1,
                         stop_words = 'english',
                         lowercase = True) #add norm="l1" or "l2" to normalize vectors
    descCorpus = []
    for n in range(num):
        f = descFolder +"/" + str(n) + ".txt"
        fileDescriptions = readDescriptionFile(f) #2D Array
        stemDescriptions = stemmedDescriptions(fileDescriptions) #applying stemming so to aggregate similar words
        for desc in stemDescriptions:
            descCorpus.append(" ".join(desc))
    return tf,tf.fit_transform(descCorpus) #TfIdfVectorizer uses unicode strings for column names.
    #return tf

In [24]:
descBag = createBag(numTrain,descriptionTrainFolder)
indexedDescBag = indexBag(descBag)

In [25]:
descVectors = binaryVectorizeDescriptions(numTrain,descriptionTrainFolder,indexedDescBag)

In [26]:
normalizedDescVector = descVectors.div(descVectors.sum(axis=1), axis=0)

In [27]:
print (normalizedDescVector.shape)

(10000, 6456)


# reducing dimensions of Image Features

In [28]:
def reduceDimensions(x,ncomp):
    #ncomp = number of components
    pca = PCA(ncomp)
    pca.fit(x)
    newX = pca.transform(x)
    return newX

In [29]:
imageDF = result
del imageDF[0]
del imageDF["index"]

In [30]:
imageMatrix = imageDF.values

In [31]:
imageMatrix[0:2]

array([[-4.55291319, -3.33937263,  0.73980451, ..., -4.32532072,
        -1.36100817, -0.1848945 ],
       [ 1.05579996,  1.34591961, -4.62148428, ..., -0.33549014,
         4.46938419, -0.19354703]])

In [33]:
imagePCAArray= reduceDimensions(imageMatrix,200) #PCA rray

In [34]:
descVectors.shape , imagePCAArray.shape

((10000, 6456), (10000, 200))

In [35]:
descVectors.values

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [41]:
from sklearn.preprocessing import MultiLabelBinarizer

In [43]:
 mlb = MultiLabelBinarizer()
binImages = mlb.fit_transform(imagePCAArray)

In [44]:
binImages[0:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# SVM

In [36]:
def SVM(train,trainLabel,test):
#     clf = SVC()
#     clf.fit(train,trainLabel)
#     preds = clf.predict(test)
    preds = OneVsRestClassifier(LinearSVC(random_state=0)).fit(train, trainLabel).predict(test)
    return preds

In [37]:
def crossValidate(data,labels):
    kf = KFold(len(data),3)
    for trainIdx,testIdx in kf:
        c = 0
        train = np.array([data[idx] for idx in trainIdx])
        test = np.array([data[idx] for idx in testIdx])
        trainLabels = np.array([labels[idx] for idx in trainIdx])
        testLabels = np.array([labels[idx] for idx in testIdx])
        print (train.shape,trainLabels.shape)
        testPreds = SVM(train,trainLabels,test)
        for i in range(len(testPreds)):
            if np.array_equal(testPreds[i],testLabels[i]):
                c+=1
        print (c)
        print (float(len(testPreds)))
        print ("Accuracy: " + str(c/float(len(testPreds))))

In [ ]:
crossValidate(descVectors.values, binImages)